In [1]:
!pip install transformers
!pip install -q sentencepiece

     |████████████████████████████████| 3.8 MB 12.1 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 38.4 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 13.2 MB/s 


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import json
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus").to(device)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive  #This is to make a short form for the mydrive location 

In [ ]:
def findSummary(sentence):
    batch = tokenizer(sentence, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text[0]

In [33]:
def findSummaries(json_file_name,lb,ub):
  f = open(f'/content/{json_file_name}')
  data = json.load(f)
  companies = pd.read_csv("/content/GoodCom.csv")
  ciks = companies["CIK"].astype(int).tolist()
  years = [2021,2020]
  for cik in ciks[lb:ub]:
    for year in years:
      try:
        sentences = data[f"{cik}"][f"{year} Sentences"]
        fhand = open(f'/mydrive/SEC_8K/{cik}_{year}.txt','w+')
        for sentence in sentences:
          summary = findSummary(sentence)
          summary = summary.capitalize()
          fhand.write(summary+'\n')
        fhand.close()
      except Exception as e:
        print(e)
        continue
  print("Done")

In [34]:
json_file_name = "data_8k_text_mihir.json" #name of json file obtained from bulk scraping
lb = 0 #start of range
ub = 1 #end of range

#Create a folder called "SEC_8K" in My Drive section of Google Drive
#Upload json file obtained from scraping in the Files section of Google Collab
#Upload GoodCom.csv in the Files section of Google Collab

findSummaries(json_file_name,lb,ub)

Done
